In [265]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import f1_score,recall_score,precision_score,roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,GridSearchCV,RandomizedSearchCV
from xgboost import XGBClassifier
import xgboost as xbg


In [266]:
train = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/flight_delays_train.csv')
test = pd.read_csv('https://raw.githubusercontent.com/Yorko/mlcourse.ai/master/data/flight_delays_test.csv')

In [267]:
train

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance,dep_delayed_15min
0,c-8,c-21,c-7,1934,AA,ATL,DFW,732,N
1,c-4,c-20,c-3,1548,US,PIT,MCO,834,N
2,c-9,c-2,c-5,1422,XE,RDU,CLE,416,N
3,c-11,c-25,c-6,1015,OO,DEN,MEM,872,N
4,c-10,c-7,c-6,1828,WN,MDW,OMA,423,Y
...,...,...,...,...,...,...,...,...,...
99995,c-5,c-4,c-3,1618,OO,SFO,RDD,199,N
99996,c-1,c-18,c-3,804,CO,EWR,DAB,884,N
99997,c-1,c-24,c-2,1901,NW,DTW,IAH,1076,N
99998,c-4,c-27,c-4,1515,MQ,DFW,GGG,140,N


In [268]:
test

,Month,DayofMonth,DayOfWeek,DepTime,UniqueCarrier,Origin,Dest,Distance
0,c-7,c-25,c-3,615,YV,MRY,PHX,598
1,c-4,c-17,c-2,739,WN,LAS,HOU,1235
2,c-12,c-2,c-7,651,MQ,GSP,ORD,577
3,c-3,c-25,c-7,1614,WN,BWI,MHT,377
4,c-6,c-6,c-3,1505,UA,ORD,STL,258
...,...,...,...,...,...,...,...,...
99995,c-6,c-5,c-2,852,WN,CRP,HOU,187
99996,c-11,c-24,c-6,1446,UA,ORD,LAS,1515
99997,c-1,c-30,c-2,1509,OO,ORD,SGF,438
99998,c-1,c-5,c-5,804,DL,LGA,ATL,761


 # Предсказываем только по Distance и DepTime
 # LogisticRegression

In [269]:
X_train , y_train = train[['Distance','DepTime']].values , train['dep_delayed_15min'].map({'N':1,'Y':0}).values
X_test = test[['Distance','DepTime']].values
X_train , X_valid , y_train , y_valid = train_test_split(X_train,y_train,test_size=0.3,random_state=17)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_valid_scaled = scaler.transform(X_valid)

X_train_scaled.shape , X_valid_scaled.shape , y_train.shape , y_valid.shape

((70000, 2), (30000, 2), (70000,), (30000,))

In [270]:
logit = LogisticRegression(random_state=17).fit(X_train_scaled,y_train)
logit_pred = logit.predict_proba(X_valid_scaled)[:,1]
print('Roc auc score : %f'%roc_auc_score(y_valid,logit_pred))
print('Recall score : %f'%recall_score(y_valid,logit.predict(X_valid_scaled)))
print('Precision score : %f'%precision_score(y_valid,logit.predict(X_valid_scaled)))
print('F1 score : %f'%f1_score(y_valid,logit.predict(X_valid)))

Roc auc score : 0.679570
Recall score : 1.000000
Precision score : 0.811194
F1 score : 0.000821


 # Предсказываем только по Distance и DepTime
# xbg_model

In [271]:
dtrain = xbg.DMatrix(X_train,y_train)
dtest = xbg.DMatrix(X_valid,y_valid)

In [272]:
params = {
    'objective':'binary:logistic',
    'max_depth':2,
    'eta':1,
    'silent':1,
}

num_rounds = 500

In [273]:
xbg_model = xbg.train(params,dtrain,num_rounds)

[05:26:19] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[05:26:19] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


In [274]:
watchlist = [(dtest,'test'),(dtrain,'train')]
xbg_model = xbg.train(params,dtrain,num_rounds,watchlist)

[05:26:20] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:576: 
Parameters: { "silent" } might not be used.

  This could be a false alarm, with some parameters getting used by language bindings but
  then being mistakenly passed down to XGBoost core, or some parameter actually being used
  but getting flagged wrongly here. Please open an issue if you find any such cases.


[05:26:20] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
[0]	test-logloss:0.46593	train-logloss:0.46672
[1]	test-logloss:0.45244	train-logloss:0.45327
[2]	test-logloss:0.44845	train-logloss:0.44894
[3]	test-logloss:0.44690	train-logloss:0.44714
[4]	test-logloss:0.44666	train-logloss:0.44637
[5]	test-logloss:0.44586	train-logloss:0.44575
[6]	test-logloss:0.44592	train-logl

[156]	test-logloss:0.44503	train-logloss:0.43287
[157]	test-logloss:0.44509	train-logloss:0.43284
[158]	test-logloss:0.44508	train-logloss:0.43279
[159]	test-logloss:0.44509	train-logloss:0.43273
[160]	test-logloss:0.44516	train-logloss:0.43265
[161]	test-logloss:0.44523	train-logloss:0.43256
[162]	test-logloss:0.44519	train-logloss:0.43252
[163]	test-logloss:0.44527	train-logloss:0.43244
[164]	test-logloss:0.44522	train-logloss:0.43239
[165]	test-logloss:0.44527	train-logloss:0.43237
[166]	test-logloss:0.44526	train-logloss:0.43234
[167]	test-logloss:0.44524	train-logloss:0.43230
[168]	test-logloss:0.44524	train-logloss:0.43227
[169]	test-logloss:0.44532	train-logloss:0.43226
[170]	test-logloss:0.44532	train-logloss:0.43222
[171]	test-logloss:0.44545	train-logloss:0.43216
[172]	test-logloss:0.44550	train-logloss:0.43212
[173]	test-logloss:0.44553	train-logloss:0.43209
[174]	test-logloss:0.44553	train-logloss:0.43207
[175]	test-logloss:0.44553	train-logloss:0.43203
[176]	test-logloss:0

[324]	test-logloss:0.44695	train-logloss:0.42714
[325]	test-logloss:0.44704	train-logloss:0.42712
[326]	test-logloss:0.44706	train-logloss:0.42708
[327]	test-logloss:0.44705	train-logloss:0.42706
[328]	test-logloss:0.44704	train-logloss:0.42704
[329]	test-logloss:0.44709	train-logloss:0.42702
[330]	test-logloss:0.44711	train-logloss:0.42702
[331]	test-logloss:0.44705	train-logloss:0.42700
[332]	test-logloss:0.44703	train-logloss:0.42699
[333]	test-logloss:0.44709	train-logloss:0.42696
[334]	test-logloss:0.44706	train-logloss:0.42695
[335]	test-logloss:0.44705	train-logloss:0.42692
[336]	test-logloss:0.44703	train-logloss:0.42691
[337]	test-logloss:0.44707	train-logloss:0.42688
[338]	test-logloss:0.44710	train-logloss:0.42685
[339]	test-logloss:0.44716	train-logloss:0.42682
[340]	test-logloss:0.44719	train-logloss:0.42680
[341]	test-logloss:0.44731	train-logloss:0.42678
[342]	test-logloss:0.44736	train-logloss:0.42676
[343]	test-logloss:0.44741	train-logloss:0.42676
[344]	test-logloss:0

In [275]:
pred_xgb = xbg_model.predict(dtest)
roc_auc_score(y_valid,pred_xgb)
predicted_labels  = pred_xgb > 0.5
print('Roc auc score : %f'%roc_auc_score(y_valid,pred_xgb))
print('Recall score : %f'%recall_score(y_valid,predicted_labels))
print('Precission score : %f'%precision_score(y_valid,predicted_labels))
print('F1 score : %f'%f1_score(y_valid,predicted_labels))

Roc auc score : 0.689358
Recall score : 0.986563
Precission score : 0.820057
F1 score : 0.895637


 # Предсказываем только по Distance и DepTime
# XGBClassifier

In [276]:
xgb_clf = XGBClassifier(random_state=17,n_jobs=-1)

In [277]:
%%time
xgb_clf.fit(X_train,y_train)

[05:26:24] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


C:\Users\emely\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


Wall time: 848 ms


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=-1,
              num_parallel_tree=1, predictor='auto', random_state=17,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

In [278]:
params = {
    'max_depth':range(1,10,2),
    'min_child_weight':range(1,6,2),
    'gamma':[i/10 for i in range(0,5)],
    'n_estimators':range(100,1001,100),
    'subsample':[i/10 for i in range(6,10)],
    'colsample_bytree':[i/10 for i in range(6,10)],
    'reg_alpha':[1e-5,1e-2,0.1,1,100],
}

In [279]:
%%time
xgb_clf_random_search_cv = RandomizedSearchCV(estimator=xgb_clf,param_distributions=params,n_jobs=-1).fit(X_train,y_train)

C:\Users\emely\anaconda3\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[05:27:19] WARNING: D:\bld\xgboost-split_1634712635879\work\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Wall time: 55.1 s


In [282]:
print('Roc auc score : %f'%roc_auc_score(y_valid,xgb_clf.predict(X_valid)))
print('Recall score : %f'%recall_score(y_valid,xgb_clf.predict(X_valid)))
print('Precission score : %f'%precision_score(y_valid,xgb_clf.predict(X_valid)))
print('F1 score : %f'%f1_score(y_valid,xgb_clf.predict(X_valid)))

Roc auc score : 0.530643
Recall score : 0.988206
Precission score : 0.820779
F1 score : 0.896745
